In [69]:
import pandas as pd
df2017=pd.read_excel('bagapursedata/bagapursedata2017.xlsx',header=0)
df2017.rename(columns={df2017.columns[0]:'firstcolumn'},inplace=True)

#create column Date
df2017['Date'] =df2017['firstcolumn'].str.extract('^(\d\d \d\d)',expand=True)
df2017['Date']=df2017['Date'].ffill()
#add year 2017
df2017['Date']=df2017['Date']+'17'
#strip whitespace
def remove_whitespace(x):
    try:
        # remove spaces inside and outside of string
        x = "".join(x.split())

    except:
        pass
    return x

df2017['Date']=df2017['Date'].apply(remove_whitespace)
#convert to datetime
df2017['Date']=pd.to_datetime(df2017['Date'], dayfirst=True)

#create column dayofweek
df2017['day_of_week'] = df2017['Date'].dt.weekday_name

#create column item sold
df2017['Items']=df2017['firstcolumn']

###can do better extract of date as compared to items sold by setting an x value for date and only print when next date is found
df2017['Items']=df2017.Items.shift(-1)
Itemslist=['bo','Bo','bracelet','Bracelet','chocker','Chocker','collier','Collier','fleur','fwp','Fwp','necklace','Necklace','ring','Ring','sac','Sac','stingray','Stingray']
df2017['ItemsCat']=df2017['Items'].str.extract(r"(?=("+'|'.join(Itemslist)+r"))", expand=True)


#create column price
import re
price=re.compile('(?<!No\s)(?<!new)(?!2016)(\d{2,4})+€?')

REGEX = '(?<!No\s)(?<!new)(?!2016)(\d{2,4})+€?'
s = df2017['Items']
df2017['Price']=s.str.extract(REGEX).astype(float)
df2017['Price']

#df2017['Price']=df2017['Items'].str.extract(price, expand=True)
##(?!(2015|2016|2017)) excluding the number that is the year
#not the numeric characters after no or No(space), only want numeric characters 2 or 3 digits before esign or (space)cash or (space)carte or (space)cpr

## this function was a tryout
def extract_int_price():
    text=df2017['Items'].astype(str)
    text=text.to_string()
    amount=[int(x) for x in re.findall('(?<!No\s)(?<!new)(?!2016)(\d{2,4})+€?', text)]
    return amount
#print (extract_int_price())

#create column method
method= re.compile('(cash|carte|cpr)')
df2017['Method']=df2017['Items'].str.extract(method, expand=True)

#create column nationality
Nationality=['phi','gr','Grecs','Grecque','fr','Fr','ind','us','usa','USA','US','can','Canada','Japon','jap','Jap','argen','Argentina',
             'uruguay','chi','Chine','chinois','chin','Shanghai','sing', 'Singapour','irl','it','ita','Ita','uk','Eng', 'England','bra', 'Brazil','mar', 'maroc','columbia',
             'Luxemberg','dutch','Holland','holland','bel','Belges','aus','ausie','austr','suisse','Suisses','Suisse','taiwan','Taiwan','korean','Indonesie',
             'India','Ger','new zealand','liban','Latino','latino','esp','Esp','bul','Romania','turq','turk','Iran','rus','japan','arab','arabs','Arabes','slo','Slovenia','viet']

df2017['Nationality']=df2017['Items'].str.extract(r"(?=("+'|'.join(Nationality)+r"))", expand=True)

#^((?!hede).)*$
#^(?:(?!ab).)+$
#(?!^ABC$)(^.*$)
#nat= re.compile('(?=('+'|'.join(Nationality)+'))')
#nat='((?:!^stingray$)(^.*$))('+'|'.join(Nationality)+')' #this one output all Nan
#nat='((?!^stingray$)('+'|'.join(Nationality)+'))'

#df2017['Nationality']=df2017['Items'].str.extract(nat)

### regex only get the most left of the string while mostly the correct nationality is the one on the most right
##excluding the word 'stingray','gris' so regex don't get nationality 'gr' out of it, also it from'howlite',
##argen from'argentee', chi from 'matching'


#group different spellings of nationalities into the same key
#1.make a dictionary 1 key with multiple values
#PHI, GRE, FRA, IND, USA, CAN, ARG, CHI, SIN, IRE, ITA, UK, BRA, MAR, COL,HOL (NLD?, DUTCH?), BEL, AUS, INDO (India?), CH (not Suisse), TAI, KOR, GER, NZ, Liban, Latino, ESP, BUL, TURK?, RUS, JAP, PER, Arabs, SLO, VIET
#NationalityDict = {'PHI':['phi'],'GRE':['Grecs', 'gr' 'Grecque'], 'FRA':['fr', 'Fr'],'IND':'ind', 'USA':['us','usa', 'USA']}
NationalityDict = {'PHI':('phi', 'filipino'),'GRE':('Grecs', 'gr', 'Grecque'), 'FRA':('fr', 'Fr'),'IND':('ind', 'india','India'),
                   'USA':('us','usa', 'USA','US'),'CAN': ('can','Canada'),'JAP': ('Japon', 'jap','japan','Jap'), 'ARG': ('argen','Argentina'),
                   'URU': ('uruguay', 'uru'),'CHI': ('chi', 'Shanghai', 'chinois','Chine','chin'),'SIN': ('sing', 'singapore','Singapour'),
                   'ITA':('it','ita','Ita'),'UK': ('uk','Eng', 'England', 'UK'), 'BRA':('bra', 'Brazil'), 'MAR':('mar','maroc'),
                   'NLD': ('Holland','dutch','holland'), 'COL':('columbia', 'col'), 'BEL':('bel','Belges'), 'AUS': ('aus', 'ausie','austr'), 
                   'SWI': ('suisse', 'Suisses','Suisse'), 'TAI':('Taiwan', 'tai'), 'KOR': ('korean','Kor'), 
                   'INDO': ('Indonesie', 'indonesie'), 'GER':('ger','Ger'), 'NZ': ('NewZealand', 'new zealand'), 
                   'LIB': ('liban', 'libanon'), 'LATI': ('Latino','latino'), 'ESP':('esp', 'Esp'), 'BUL':('bul', 'Bul'), 
                   'ROM':('Romania','romania'), 'TUR':('turq','turk'), 'IRA':('Iran', 'iran'), 'RUS':('rus','Russia'), 
                   'JAP':('Japon', 'japan'), 'ARAB':('arab', 'Arabes','arabs'), 'SLO':('slo', 'Slovenia'), 'VIET':('viet','Viet')}

#2. make an inversed dictionary
rev_dict= dict((v, k) for k, v in NationalityDict.items())
rev_dict #{'phi': 'PHI', ('Grecs', 'gr', 'Grecque'): 'GRE', ('fr', 'Fr'): 'FRA','ind': 'IND',('us', 'usa', 'USA'): 'USA'}
rev_dict2= {value: key for key in NationalityDict for value in NationalityDict[key]}


#3. map nationality to inversed dict
df2017['Natsum']=df2017['Nationality'].map(rev_dict2)


#create column time
df2017['Time']=df2017['Items'].str.extract('(\d\dh\d+)', expand=True)


pd.set_option('display.max_rows',600)

#sum price per nationality
#nationalityhigh=df2017.groupby('Natsum', as_index=False)['Price'].sum()
nationalityhigh=df2017.groupby('Natsum')['Price'].sum()
nationalityhigh

#pcs sold per nationality
nationalitypcshigh=df2017.groupby('Natsum')['Items'].count()
nationalitypcshigh #Natsum ARG    14 CAN     3 FRA    17 GRE    94 ITA    62 UK     12 USA    31 Name: Items, dtype: int64

#sum price per day of the week
#dayoftheweekhigh=df2017.groupby('day_of_week')['Price'].sum()
#dayoftheweekhigh
#day_of_week Friday        8512.0 Monday       33219.0 Saturday     14202.0 Sunday       12580.0 Thursday     15793.0 Tuesday       5614.0 Wednesday     6803.0 Name: Price, dtype: float64

#categorize afternoon, evening, night, then sum price per category

#df2017[['Items','Nationality','Natsum']]
#df2017[df2017['Items'].str.contains('stingray')==True]

from pandas import ExcelWriter
writer=ExcelWriter('df2017.xlsx')
df2017.to_excel(writer,'sheet1')
writer.save()

df2017[['Items','Nationality', 'Natsum']]
nationalityhigh
df2017.groupby('ItemsCat').count()

C:\Users\Pimwipa\Anaconda3\lib\site-packages\ipykernel\__main__.py:42: FutureWarning: currently extract(expand=None) means expand=False (return Index/Series/DataFrame) but in a future version of pandas this will be changed to expand=True (return DataFrame)


,firstcolumn,Date,day_of_week,Items,Price,Method,Nationality,Natsum,Time
ItemsCat,,,,,,,,,
Bo,79,79,79,79,79,78,60,60,68
Bracelet,23,23,23,23,23,23,21,21,15
Chocker,60,60,60,60,60,60,55,55,52
Collier,36,36,36,36,36,36,32,32,30
Fwp,15,15,15,15,15,14,14,14,12
Necklace,44,44,44,44,43,42,31,31,34
Ring,43,43,43,43,42,40,35,35,36
Sac,3,3,3,3,3,2,2,2,3
Stingray,22,22,22,22,22,22,22,22,16
